In [ ]:
import os
# os.environ['DEVICE_ID'] = '0'
import numpy as np

import mindspore as ms
from mindspore import nn
from mindspore import context

# Ascend

# 1.导入MindSpore模块和辅助模块
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")

# 2.生成模拟数据
x = np.arange(-5, 5, 0.3)[:32].reshape((32, 1))
y = -5 * x + 0.1 * np.random.normal(loc=0.0, scale=20.0, size=x.shape)

# 3.建模
net = nn.Dense(1, 1)
loss_fn = nn.loss.MSELoss()
opt = nn.optim.SGD(net.trainable_params(), learning_rate=0.01)
with_loss = nn.WithLossCell(net, loss_fn)
train_step = nn.TrainOneStepCell(with_loss, opt).set_train()

# 4.使用模拟数据训练模型
for epoch in range(20):
    loss = train_step(ms.Tensor(x, ms.float32), ms.Tensor(y, ms.float32))
    print('epoch: {0}, loss is {1}'.format(epoch, loss))

# 5.使用训练好的模型进行预测
wb = [x.asnumpy() for x in net.trainable_params()]
w, b = np.squeeze(wb[0]), np.squeeze(wb[1])
print('The true linear function is y = -5 * x + 0.1')
print('The trained linear model is y = {0} * x + {1}'.format(w, b))

for i in range(-10, 11, 5):
    print('x = {0}, predicted y = {1}'.format(i, net(ms.Tensor([[i]], ms.float32))))

# 6.可视化
from matplotlib import pyplot as plt
%matplotlib inline
plt.scatter(x, y, label='Samples')
plt.plot(x, w * x + b, c='r', label='True function')
plt.plot(x, -5 * x + 0.1, c='b', label='Trained model')
plt.legend()